In [2]:
import os
import requests

# Tạo thư mục 'poker' nếu chưa tồn tại
if not os.path.exists('poker'):
    os.makedirs('poker')

# Các chất bài (tép, rô, cơ, bích)
suits = ['tep', 'ro', 'co', 'bich']

# Giá trị của lá bài từ K (13) xuống A (1)
values = {13: 'K', 12: 'Q', 11: 'J', 10: '10', 9: '9', 8: '8', 7: '7', 6: '6', 5: '5', 4: '4', 3: '3', 2: '2', 1: 'A'}

# Đọc file chứa đường dẫn các lá bài
with open('./link_bai.txt', 'r') as f:
    links = f.readlines()

# Kiểm tra xem số đường dẫn có đủ 52 lá bài hay không
if len(links) != 52:
    raise ValueError("Số lượng đường dẫn không đủ 52 lá bài!")

# Tải từng lá bài về theo thứ tự giá trị và chất
index = 0
for value in range(13, 0, -1):  # Từ K (13) đến A (1)
    for suit in suits:
        # Đường dẫn ảnh
        link = links[index].strip()
        # Tên file ảnh theo giá trị và chất (ví dụ: K_tep.jpg)
        filename = f'{values[value]}_{suit}.jpg'
        # Đường dẫn lưu vào thư mục 'poker'
        filepath = os.path.join('poker', filename)

        # Tải ảnh về
        response = requests.get(link)
        if response.status_code == 200:
            # Ghi file ảnh
            with open(filepath, 'wb') as img_file:
                img_file.write(response.content)
            print(f"Tải thành công: {filename}")
        else:
            print(f"Không thể tải ảnh từ {link}")

        index += 1


Tải thành công: K_tep.jpg
Tải thành công: K_ro.jpg
Tải thành công: K_co.jpg
Tải thành công: K_bich.jpg
Tải thành công: Q_tep.jpg
Tải thành công: Q_ro.jpg
Tải thành công: Q_co.jpg
Tải thành công: Q_bich.jpg
Tải thành công: J_tep.jpg
Tải thành công: J_ro.jpg
Tải thành công: J_co.jpg
Tải thành công: J_bich.jpg
Tải thành công: 10_tep.jpg
Tải thành công: 10_ro.jpg
Tải thành công: 10_co.jpg
Tải thành công: 10_bich.jpg
Tải thành công: 9_tep.jpg
Tải thành công: 9_ro.jpg
Tải thành công: 9_co.jpg
Tải thành công: 9_bich.jpg
Tải thành công: 8_tep.jpg
Tải thành công: 8_ro.jpg
Tải thành công: 8_co.jpg
Tải thành công: 8_bich.jpg
Tải thành công: 7_tep.jpg
Tải thành công: 7_ro.jpg
Tải thành công: 7_co.jpg
Tải thành công: 7_bich.jpg
Tải thành công: 6_tep.jpg
Tải thành công: 6_ro.jpg
Tải thành công: 6_co.jpg
Tải thành công: 6_bich.jpg
Tải thành công: 5_tep.jpg
Tải thành công: 5_ro.jpg
Tải thành công: 5_co.jpg
Tải thành công: 5_bich.jpg
Tải thành công: 4_tep.jpg
Tải thành công: 4_ro.jpg
Tải thành công: 4_

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
from PIL import Image

# Đường dẫn đến thư mục chứa hình ảnh bài
data_dir = './poker'

# Tạo thư mục lưu trữ ảnh đã tăng cường
augmented_dir = './poker_augmented'
if not os.path.exists(augmented_dir):
    os.makedirs(augmented_dir)

# Các thông số tăng cường dữ liệu
datagen = ImageDataGenerator(
    rotation_range=30,      # Xoay ngẫu nhiên trong khoảng 30 độ
    width_shift_range=0.1,  # Dịch chuyển theo chiều ngang
    height_shift_range=0.1, # Dịch chuyển theo chiều dọc
    shear_range=0.2,        # Biến đổi hình ảnh theo góc xiên
    zoom_range=0.2,         # Phóng to hoặc thu nhỏ ngẫu nhiên
    horizontal_flip=True,   # Lật ngang
    brightness_range=(0.5, 1.5)  # Thay đổi độ sáng
)

# Hàm để tạo tên thư mục theo lá bài (VD: 2_tep, K_co, A_bich)
def get_card_folder_name(filename):
    # Giả sử tên file là "K_co.jpg" hoặc "2_tep.jpg"
    card_name = filename.split('.')[0]  # Bỏ phần đuôi ".jpg"
    return card_name

# Duyệt qua tất cả các hình ảnh trong thư mục gốc
for filename in os.listdir(data_dir):
    if filename.endswith('.jpg'):
        img_path = os.path.join(data_dir, filename)
        img = Image.open(img_path)
        img = img.resize((150, 150))  # Resize về kích thước nhỏ hơn cho nhanh

        # Chuyển đổi hình ảnh sang chế độ RGB nếu cần thiết
        if img.mode == 'RGBA':
            img = img.convert('RGB')

        # Tạo thư mục con theo lá bài (VD: 2_tep, A_co)
        card_folder_name = get_card_folder_name(filename)
        card_folder_path = os.path.join(augmented_dir, card_folder_name)
        
        # Tạo thư mục nếu chưa tồn tại
        if not os.path.exists(card_folder_path):
            os.makedirs(card_folder_path)

        # Chuyển hình ảnh sang numpy array
        x = np.array(img)
        x = x.reshape((1,) + x.shape)  # Thêm batch dimension

        # Tạo ra 10 phiên bản ảnh từ một ảnh gốc và lưu vào thư mục con tương ứng
        i = 0
        for batch in datagen.flow(x, batch_size=1, save_to_dir=card_folder_path, save_prefix=filename.split('.')[0], save_format='jpg'):
            i += 1
            if i >= 10:  # Tạo 10 ảnh tăng cường cho mỗi quân bài
                break

print("Tăng cường dữ liệu hoàn tất!")


Tăng cường dữ liệu hoàn tất!


In [6]:
# Đường dẫn mới đến tập dữ liệu đã được tăng cường
augmented_data_dir = './poker_augmented'

# Tạo lại trình tạo dữ liệu cho việc huấn luyện và xác thực
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    augmented_data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    augmented_data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Xây dựng mô hình như trước đó
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Huấn luyện mô hình
model.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator
)

# Lưu mô hình
model.save('card_recognition_augmented_model.h5')


Found 832 images belonging to 52 classes.
Found 207 images belonging to 52 classes.
Epoch 1/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 7s 213ms/step - accuracy: 0.0322 - loss: 4.0208 - val_accuracy: 0.0483 - val_loss: 3.7264
Epoch 2/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 202ms/step - accuracy: 0.0751 - loss: 3.4737 - val_accuracy: 0.0966 - val_loss: 3.1314
Epoch 3/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 198ms/step - accuracy: 0.2094 - loss: 2.6334 - val_accuracy: 0.1256 - val_loss: 3.1291
Epoch 4/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 196ms/step - accuracy: 0.4010 - loss: 1.9987 - val_accuracy: 0.1256 - val_loss: 3.3253
Epoch 5/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 195ms/step - accuracy: 0.6186 - loss: 1.2374 - val_accuracy: 0.1111 - val_loss: 4.1882
Epoch 6/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 196ms/step - accuracy: 0.8120 - loss: 0.6844 - val_accuracy: 0.1401 - val_loss: 5.1651
Epoch 7/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 194ms/step - accuracy: 0.8581 - loss: 0.4126 - val_accuracy: 0.1353 - val_loss: 5.9323
Epoch 8/30
26/26 ━━━

In [10]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Đọc ảnh mới và dự đoán
img_path = './poker/j_bich.jpg'
img = image.load_img(img_path, target_size=(150, 150))  # Kích thước ảnh mà mô hình được huấn luyện
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Thêm chiều batch dimension

# Dự đoán kết quả
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions, axis=1)

# Lấy tên class từ các thư mục đã dùng để train
class_names = list(train_generator.class_indices.keys())
print(f"Mô hình dự đoán: {class_names[predicted_class[0]]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Mô hình dự đoán: J_bich
